# Overview

We need to refine the way users specify colors, to simplify the public API and eliminate inconsistencies.  The challenge is to handle all of the ways in which a user might want to specify per-series / per-datum colors, while avoiding potential ambiguities.

**Use-cases**

* Specify one explicit color for everything.
        toyplot.plot(x, y, color="red")
* Specify an explicit color per series.
        toyplot.plot(x, y, color=["red", "cinnamon", "cardinal"])
* Specify a palette for the series.
        toyplot.plot(x, y, color=toyplot.color.brewer("reds"))
* Specify a colormap for the series.
        toyplot.plot(x, y, color=toyplot.color.LinearMap(toyplot.color.brewer("reds")))
* Specify per-datum scalar values and have them mapped.
        toyplot.plot(x, y, color=temperature)
* Specify per-datum scalar values and have them mapped, but override a few "special" values.
        colors = toyplot.color.broadcast(temperature)
        colors[numpy.argsort(temperature)[0]] = "blue"
        colors[numpy.argsort(temperature)[-1]] = "red"
        toyplot.plot(x, y, color=colors)
* Specify explicit per-datum color values for everything (i.e. map them yourself).
        toyplot.plot(x, y, color=mycolors)

**Mark Colors**

* bars -> color, fill?, stroke
* fill -> color, fill?, stroke?
* graph -> color?, vcolor, vfill, vstroke, vlfill, vlstroke, ecolor, elfill, elstroke
* hlines -> color, stroke?
* plot -> color, stroke, mfill, mstroke, mlfill, mlstroke
* rects -> color, fill, stroke
* scatterplot -> color, mfill, mstroke, mlfill, mlstroke
* vlines -> color, stroke?

**toyplot.color.broadcast(...)**

* Always returns a numpy color array (array with dtype == toyplot.color.dtype), with the given shape.

*Implemented*

* color=None, shape=$M \times 1$ - Use a single default color.
* color=values, shape=$M \times 1$ - Map values with a linear colormap and a default palette.
* color=colormap, shape=$M \times 1$ - Map the range $[0, M)$ with the given colormap.
* color=palette, shape=$M \times 1$ - Map the range $[0, M)$ with a linear colormap and the given palette.
* color=(values, colormap), shape=$M \times 1$ - Map values with the given colormap.
* color=(values, palette), shape=$M \times 1$ - Map values with a linear colormap and the given palette.

*Not Implemented*

* color=None, shape=$M$ -
* color=values, shape=$M$ -
* color=colormap, shape=$M$ - 
* color=palette, shape=$M$ -
* color=(values, colormap), shape=$M$ - map values with the given colormap.
* color=(values, palette), shape=$M$ - map values with a linear colormap and the given palette.


* color=None, shape=$M \times N$ -
* color=values, shape=$M \times N$ -
* color=colormap, shape=$M \times N$ - 
* color=palette, shape=$M \times N$ -
* color=(values, colormap), shape=$M \times N$ - map values with the given colormap.
* color=(values, palette), shape=$M \times N$ - map values with a linear colormap and the given palette.

**Values**

* <strike>"value" - a single CSS color value.</strike>
* <strike>(r, g, b) - a single RGB color value.</strike>
* <strike>(r, g, b, a) - a single RGBA color value.</strike>
* <strike>toyplot.color.rgb() - a single RGB color value.</strike>
* <strike>toyplot.color.rgba() - a single RGBA color value.</strike>
* <strike>[value1, value2, ...] - 1D heterogeneous collection of CSS and tuple color values.</strike>
    * We don't allow scalar values in this case, because their domain would be ambiguous.
    * Have to be careful with the implementation here - we don't want a list of nothing-but-tuples to be misinterpreted as a 2D numpy array of scalars.
* <strike>numpy string array with shape $M$ - 1D collection of CSS color values.</strike>
* numpy string array with shape $M \times N$ - 2D collection of CSS color values.
* <strike>numpy numeric array with shape $M$ - 1D collection of scalar values for mapping.</strike>
* <strike>numpy numeric array with shape $M \times N$ - 2D collection of scalar values for mapping.</strike>
* <strike>numpy color array with shape $M$ - 1D collection of color values.</strike>
* numpy color array with shape $M \times N$ - 2D collection of color values.

In [1]:
import numpy
x = numpy.linspace(0, 1)
y = numpy.column_stack((x ** 2, x ** 3))
numpy.random.seed(1234)
temperature = numpy.random.uniform(size=y.shape)

In [2]:
import logging
import toyplot
toyplot.log.level = logging.DEBUG

In [3]:
series = numpy.column_stack((numpy.linspace(0, 1), numpy.linspace(0, 1) ** 2))

In [4]:
toyplot.bars(series, color=None, width=300);

DEBUG:toyplot:broadcast (2,) to (50, 2)


In [5]:
toyplot.bars(series, color=toyplot.color.brewer("Set1"), width=300);
toyplot.color.brewer("Set1")

DEBUG:toyplot:broadcast (2,) to (50, 2)


In [6]:
toyplot.bars(series, color=toyplot.color.LinearMap(toyplot.color.brewer("GreenYellowRed")), width=300);
toyplot.color.brewer("GreenYellowRed")

DEBUG:toyplot:broadcast (2,) to (50, 2)


In [7]:
toyplot.scatterplot(x, y, color=None, width=300);

DEBUG:toyplot:broadcast (2,) to 2
DEBUG:toyplot:broadcast (2,) to (50, 2)
DEBUG:toyplot:broadcast (50, 2) to (50, 2)


In [8]:
toyplot.scatterplot(x, y, color=None, width=300);

DEBUG:toyplot:broadcast (2,) to 2
DEBUG:toyplot:broadcast (2,) to (50, 2)
DEBUG:toyplot:broadcast (50, 2) to (50, 2)


In [9]:
toyplot.scatterplot(x, y, color="red", width=300);

DEBUG:toyplot:broadcast () to 2
DEBUG:toyplot:broadcast (2,) to (50, 2)
DEBUG:toyplot:broadcast (50, 2) to (50, 2)


In [10]:
toyplot.scatterplot(x, y, color=["red", "blue"], width=300);

DEBUG:toyplot:broadcast (2,) to 2
DEBUG:toyplot:broadcast (2,) to (50, 2)
DEBUG:toyplot:broadcast (50, 2) to (50, 2)


In [11]:
toyplot.scatterplot(x, y, color=toyplot.color.brewer("Set1"), width=300);

DEBUG:toyplot:broadcast (2,) to 2
DEBUG:toyplot:broadcast (2,) to (50, 2)
DEBUG:toyplot:broadcast (50, 2) to (50, 2)


In [12]:
toyplot.scatterplot(x, y, fill=temperature, width=300);

DEBUG:toyplot:broadcast (2,) to 2
DEBUG:toyplot:broadcast (50, 2) to (50, 2)
DEBUG:toyplot:broadcast (50, 2) to (50, 2)


In [13]:
toyplot.scatterplot(x, y, fill=(temperature, toyplot.color.brewer("BlueRed")), width=300);

DEBUG:toyplot:broadcast (2,) to 2
DEBUG:toyplot:broadcast (50, 2) to (50, 2)
DEBUG:toyplot:broadcast (50, 2) to (50, 2)


In [14]:
toyplot.scatterplot(x, y, fill=(temperature, toyplot.color.LinearMap()), width=300);

DEBUG:toyplot:broadcast (2,) to 2
DEBUG:toyplot:broadcast (50, 2) to (50, 2)
DEBUG:toyplot:broadcast (50, 2) to (50, 2)


In [15]:
colors = toyplot.color.broadcast((temperature, toyplot.color.brewer("Greys")), y.shape)
colors[20:30, 1] = toyplot.color.css("yellow")
colors.flat[numpy.argmin(temperature)] = toyplot.color.css("blue")
colors.flat[numpy.argmax(temperature)] = toyplot.color.css("red")
toyplot.scatterplot(x, y, fill=colors, width=300);

DEBUG:toyplot:broadcast (50, 2) to (50, 2)
DEBUG:toyplot:broadcast (2,) to 2
DEBUG:toyplot:broadcast (50, 2) to (50, 2)
DEBUG:toyplot:broadcast (50, 2) to (50, 2)


In [16]:
x = numpy.linspace(0, 1)
y = x ** 2

In [17]:
toyplot.bars(y, color=(x, toyplot.color.brewer("Reds")), width=300);

DEBUG:toyplot:broadcast (50,) to (50, 1)


In [18]:
toyplot.bars(y, color=toyplot.color.brewer("Reds"), width=300);

DEBUG:toyplot:broadcast (50,) to (50, 1)


In [19]:
toyplot.fill(x, numpy.column_stack((y, y/2, y/3, y/4, y/5)), baseline="symmetric", color=(numpy.arange(5), toyplot.color.brewer("Reds")), width=300);

DEBUG:toyplot:broadcast (5,) to 5


In [20]:
toyplot.fill(x, numpy.column_stack((y, y/2, y/3, y/4, y/5)), baseline="symmetric", color=toyplot.color.brewer("Reds"), width=300);

DEBUG:toyplot:broadcast (5,) to 5


In [21]:
toyplot.scatterplot(x, numpy.column_stack((y, y/2, y/3, y/4, y/5)), color=(numpy.arange(5), toyplot.color.brewer("Reds")), width=300);

DEBUG:toyplot:broadcast (5,) to 5
DEBUG:toyplot:broadcast (5,) to (50, 5)
DEBUG:toyplot:broadcast (50, 5) to (50, 5)


In [22]:
toyplot.scatterplot(x, numpy.column_stack((y, y/2, y/3, y/4, y/5)), color=toyplot.color.brewer("Reds"), width=300);

DEBUG:toyplot:broadcast (5,) to 5
DEBUG:toyplot:broadcast (5,) to (50, 5)
DEBUG:toyplot:broadcast (50, 5) to (50, 5)


In [23]:
canvas = toyplot.Canvas(width=300)
axes = canvas.axes()
axes.text(x, y, y, color=(x, toyplot.color.brewer("Reds")));

DEBUG:toyplot:broadcast (50,) to (50, 1)


In [24]:
canvas = toyplot.Canvas(width=300)
axes = canvas.axes()
axes.text(x, y, y, color=toyplot.color.brewer("Reds"));

DEBUG:toyplot:broadcast (50,) to (50, 1)
